In [1]:
import os
import sys
import re
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# -------------- NESTED PATH CORRECTION -------------------------------- #

# For all script files, we add the parent directory to the system path
cwd = re.sub(r"[\\]", "/", os.getcwd())
cwd_list = cwd.split("/")
path = sys.argv[0]
path_list = path.split("/")
# either the entire filepath is entered as command i python
if cwd_list[0:3] == path_list[0:3]:
    full_path = path
# or a relative path is entered, in which case we append the path to the cwd_path
else:
    full_path = cwd + "/" + path
# remove the overlap
root_dir = re.search(r"(^.+python_road_upgrades)", full_path).group(1)
sys.path.append(root_dir)

# ---------------------------------------------------------------------- #

In [2]:
def analyze_roads(pct):

    def dummy_row_distance(df, dummy, agg_list):
        if not isinstance(agg_list, list):
            raise ValueError("agg_list argument needs to be a list")
        output_index = df.reset_index().set_index([('road', ''), ('subroad', '')])
        filtered = df.loc[df[dummy] == 1,]
        dist = filtered.reset_index().loc[:, [('road', ''), ('subroad', ''), ('id', '')]].groupby(['road', 'subroad']).diff().fillna(0).values
        dist = pd.DataFrame(dist, index=filtered.index).reset_index().rename(columns={0: "dist"})
        dist = dist.reset_index()#.group_by(['road', 'subroad'])
        dist = dist.groupby(['road', 'subroad']).agg({'dist': agg_list})
        dist = dist.reindex(df.index.droplevel(2).unique())
        return dist

    folder_name = "composites_2"        # directory should exist in Exports/Roads and Exports/Points


    dir_roads = f"{root_dir}/Exports/Roads/{folder_name}"
    dir_points = f"{root_dir}/Exports/Points/{folder_name}"

    files_roads = os.listdir(dir_roads)
    files_points = os.listdir(dir_points)


    #for r in files_roads:
    for r in ["roads_2020_2021_05_07.xlsx"]:
        # r = "roads_2020_2021_05_07.xlsx"
        if r[0:5] != 'roads':
            continue
        m = re.search(r"roads_(\d+)_(\d+)_(\d+)_(\d+).xlsx", r)
        year1, year2, month1, month2 = m.group(1), m.group(2), m.group(3), m.group(4)

        # load in the roads excel file
        fp_r = f"{dir_roads}/{r}"
        df_r = pd.read_excel(fp_r, engine='openpyxl')

        # load in the csv points files and merge
        df_p = pd.DataFrame()
        for p in files_points:
            if r[:-5] in p:
                fp_p = f"{dir_points}/{p}"
                imported = pd.read_csv(fp_p)
                if df_p.empty:
                    df_p = imported
                else:
                    df_p = pd.concat([df_p, imported])
        df_p = df_p.sort_values(by=["road", "subroad", "type", "id"])


        # reshape
        wide = pd.pivot_table(df_p, values=["B4_max", "B4_mean", "B4_min", "B8_max", "B8_mean", "B8_min", "B4_diff_factor",
                                            "B8_diff_factor" ], index=['road', 'subroad', 'id'], columns=['type'], aggfunc=np.sum)

        # adjust for the diff factor
        level1 = ['B4_max', 'B4_min', 'B4_mean', 'B8_max', 'B8_min', 'B8_mean']
        level2 = ['road', 'left_25', 'left_50', 'left_75', 'right_25', 'right_50', 'right_75']
        for l1 in level1:
            if l1[0:2] == "B4":
                diff_factor_array = wide.loc[:, ('B4_diff_factor', 'road')]
            elif l1[0:2] == "B8":
                diff_factor_array = wide.loc[:, ('B8_diff_factor', 'road')]
            else:
                raise ValueError("problem with diff factor adjustment")
            for l2 in level2:
                wide[(l1, l2)] = wide[(l1, l2)] - diff_factor_array
                #pass

        # rewrite the sides as a difference from the road and
        # create the averages
        level1 = ['B4_max', 'B4_min', 'B4_mean', 'B8_max', 'B8_min', 'B8_mean']
        for l1 in level1:
            # make sides relative
            # wide[[(l1, 'left_25'), (l1, 'left_50'), (l1, 'left_75')]] = wide[[(l1, 'left_25'), (l1, 'left_50'), (l1, 'left_75')]] - wide[[(l1, 'road')]].values
            # create the averages
            wide[(l1, 'left_avg')] =  wide[[(l1, 'left_25'), (l1, 'left_50'), (l1, 'left_75')]].mean(axis=1)
            wide[(l1, 'right_avg')] =  wide[[(l1, 'right_25'), (l1, 'right_50'), (l1, 'right_75')]].mean(axis=1)


        # thresholds, why not?
        # compute upgraded points
        thresholds = [50]

        for thresh in thresholds:
            level1 = ['B4_max', 'B8_max']
            upg = f"upg_{thresh}"
            for l1 in level1:
                wide[(l1, upg)] = ( (wide[(l1, 'road')] > thresh) &
                                    (wide[(l1, 'left_avg')] <= (pct/100) * wide[(l1, 'road')]) &
                                    (wide[(l1, 'right_avg')] <= (pct/100) * wide[(l1, 'road')]) ).astype(int)
            level1 = ['B4_min', 'B8_min']
            for l1 in level1:
                wide[(l1, upg)] = ( (wide[(l1, 'road')] < -thresh) &
                                    (wide[(l1, 'left_avg')] >= (pct/100) * wide[(l1, 'road')]) &
                                    (wide[(l1, 'right_avg')] >= (pct/100) * wide[(l1, 'road')]) ).astype(int)

        upg_list = [f"upg_{x}" for x in thresholds]

        # reorder columns
        col_index_list = []
        level1 = ['B4_max', 'B4_min', 'B4_mean', 'B8_max', 'B8_min', 'B8_mean', 'B4_diff_factor', 'B8_diff_factor']
        level2 = upg_list+['road', 'left_avg', 'right_avg', 'left_25', 'left_50', 'left_75', 'right_25', 'right_50', 'right_75']
        for l1 in level1:
            for l2 in level2:
                if l1 in ['B4_diff_factor', 'B8_diff_factor'] and l2 != "road":
                    continue
                col_index_list.append((l1, l2))
        wide = wide.reindex(columns=col_index_list)

        # clean up by dropping NaN columns
        wide = wide.dropna(axis=1, how='all')



        # compute the total count in each subroad
        point_count = wide.reset_index().groupby(['road', 'subroad']).agg({('id', ''): 'count'}).values


        # generate the tuples for aggregation
        tup_dic = {}
        for l1 in ['B4_max', 'B4_min', 'B4_mean', 'B8_max', 'B8_min', 'B8_mean']:
            for l2 in ['road', 'left_avg', 'right_avg', f'upg_{thresholds[0]}']:
                if (l1, l2) in list(wide.columns):
                    tup_dic[(l1, l2)] = 'mean'
                else:
                    print(f"skipping {(l1, l2)}")

        tup_dic[('B4_diff_factor', 'road')] = 'mean'
        tup_dic[('B8_diff_factor', 'road')] = 'mean'

        brightness = wide.reset_index().groupby(['road', 'subroad']).agg(tup_dic)

        # rename the upg_0 to share
        brightness = brightness.rename(columns={f'upg_{thresholds[0]}': "share"})


        for l1 in ['B4_max', 'B4_min', 'B8_max', 'B8_min']:

            # aggregate the road, left and right avg of upgraded roads only
            upgraded_subset = wide.loc[wide[(l1, f'upg_{thresholds[0]}')] == 1, :]
            tup_dic = {}
            for l2 in ['road', 'left_avg', 'right_avg']:
                tup_dic[(l1, l2)] = 'mean'
            subset_aggs = upgraded_subset.reset_index().groupby(['road', 'subroad']).agg(tup_dic)
            # apply the complete index
            subset_aggs = subset_aggs.reindex(brightness.index)
            # insert the new aggregations into the brightness df
            col_id = list(brightness.columns).index((l1, 'right_avg'))
            for c in list(reversed(list(subset_aggs.columns))):
                brightness.insert((col_id + 1),(l1, f'{c[1]}_upg'), subset_aggs[c])

            # get the id of the share
            cols = list(brightness.columns)
            col_id = cols.index((l1, 'share'))
            dummy_aggs = ['median', 'mean']
            dummy_vars = dummy_row_distance(wide, (l1, f'upg_{thresholds[0]}'), dummy_aggs)
            for a in dummy_aggs:
                buffer = dummy_aggs.index(a)
                brightness.insert((col_id + buffer),(l1, f'dist_{a}'), dummy_vars[('dist', a)])

        brightness = brightness.sort_values(by=('B4_min', 'road'), ascending=True)  # shows the percentage of points that meet condition
        brightness = brightness.sort_index()


        cols_before_dummies = list(brightness.columns)

        # determine if the road is upgraded


        brightness[('upgrade', "bright_B4_diff")] = ( (brightness[('B4_min', 'road')] < 0 ) &
                                                      (brightness[('B4_min', 'left_avg')] > 0.75 * brightness[('B4_min', 'road')]) &
                                                      (brightness[('B4_min', 'right_avg')] > 0.75 * brightness[('B4_min', 'road')]) #&
                                                      #(brightness[('B4_min', 'left_avg')] > brightness[('B4_min', 'road')] + 50) &
                                                      #(brightness[('B4_min', 'right_avg')] > brightness[('B4_min', 'road')] + 50)
                                                      ).astype(int)


        b4_diff_array = brightness[('B4_diff_factor', 'road')].values
        brightness[('upgrade', "bright_B4")] = ( (brightness[('B4_min', 'road')] + b4_diff_array < 0 ) &
                                                 (brightness[('B4_min', 'left_avg')] + b4_diff_array > 0.75 * (brightness[('B4_min', 'road')]+ b4_diff_array)) &
                                                 (brightness[('B4_min', 'right_avg')] + b4_diff_array > 0.75 * (brightness[('B4_min', 'road')]+ b4_diff_array)) #&
                                                 #(brightness[('B4_min', 'left_avg')] - b4_diff_array > brightness[('B4_min', 'road')] - b4_diff_array + 40) &
                                                 #(brightness[('B4_min', 'right_avg')] - b4_diff_array > brightness[('B4_min', 'road')] - b4_diff_array + 40)
                                                 ).astype(int)


        cols = [('upgrade', "bright_B4"), ('upgrade', "bright_B4_diff")] + cols_before_dummies
        brightness = brightness.loc[:, cols]

        # add the values to the export sheet

        brightness_results = brightness.reset_index().rename(columns={"road": "road_id", "subroad": "subroad_id"}).set_index(['road_id', 'subroad_id'])
        #brightness_results

        # x = df_r.set_index(['road_id', 'subroad_id'])
        #
        # x['upgraded'] = brightness_results[('upgrade', 'bright_B4_diff')]
        # x['b_share_B4'] = brightness_results[('B4_min', 'road_id')]
        # x['b_share_B4_left'] = brightness_results[('B4_min', 'left_avg')]
        # x['b_share_B4_right'] = brightness_results[('B4_min', 'right_avg')]
        #
        # x = x.reset_index()


        # export the brightness results (the analysis)
        # EXPORT FILEPATH TO THE IMPORTS DIRECTORY
        analysis_filename = r.replace('roads', 'analysis')
        writer = pd.ExcelWriter(f"{root_dir}/Exports/Roads/composites_2/{analysis_filename}", engine='xlsxwriter', options={'strings_to_urls': False})
        brightness_results.reset_index().to_excel(writer)
        writer.close()

        print(f"HURRA! We finished spreadsheet {r}")


        #writer = pd.ExcelWriter("detailed_point_medians.xlsx", engine='xlsxwriter', options={'strings_to_urls': False})
        #brightness_results.reset_index().to_excel(writer)
        #riter.close()


In [3]:
def get_regressors(df):
    return df[[
        ('B4_min', 'road_id'),
        ('B4_min', 'left_avg'),
        ('B4_min', 'right_avg'),
        ('B4_mean', 'road_id'),
        #('B4_mean', 'left_avg'),
        #('B4_mean', 'right_avg'),
        #('B4_min', 'dist_median'),
        ('B4_min', 'dist_mean'),
        ('B4_min', 'share'),
        ('B4_diff_factor', 'road_id')

    ]]


In [4]:
def eval_heuristic(fp, t_size):

    # first the manual visual identifications (true values)
    visual = pd.read_excel(f'{root_dir}/Imports/visual_classification_2020_2021_05_07.xlsx').set_index(['road_id', 'subroad_id'])
    visual[visual['upgrade'] != 1] = 0  # replace unclear and NaN with 0

    # followed by the analysis data
    def import_analysis(data_fp):
        data = pd.read_excel(data_fp,  header=[0,1], index_col=[0]).set_index(
            [('road_id', 'Unnamed: 1_level_1'), ('subroad_id', 'Unnamed: 2_level_1')])
        data.index.names = ['road_id', "subroad_id"]
        data.columns.names = [None, None]
        data = data.drop([('upgrade', 'bright_B4_diff'), ('upgrade', 'bright_B4')], axis=1)
        return data

    data = import_analysis(fp)

    #print(data.columns)
    # 1.2 Split the data into training and test sets


    X = get_regressors(data)
    y = visual['upgrade'].astype('int')  # Labels


    # check for NaN and drop
    nan_index = X[X.isnull().any(1)].index
    X = X.drop(nan_index, axis=0)
    y = y.drop(nan_index, axis=0)

    # Split dataset into training set and test set
    #     - argument test_size = 0.3: 70% training and 30% test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=t_size, random_state=5350)


    # compute the optimal share threshold for the training data
    accuracy_dic = {}
    for t in range(0, 101):
        share = t/100
        h_pred = (
            (X_train[('B4_min', 'share')] > share)
        ).astype('int')
        a = metrics.accuracy_score(y_train, h_pred)
        if a not in list(accuracy_dic.keys()):
            accuracy_dic[a] = [share]
        else:
            accuracy_dic[a].append(share)

    max_key = max(list(accuracy_dic.keys()))
    print("MAX ACCURACY FIT TRAINING DATA", max_key)
    print("THRESH IS", accuracy_dic[max_key])

    h_pred_share_test = (
        (X_test[('B4_min', 'share')] > min(accuracy_dic[max_key]))
    ).astype('int')


    print("Heuristic Model Accuracy:", metrics.accuracy_score(y_test, h_pred_share_test))
    #print("Random Forest Model Accuracy:", metrics.accuracy_score(y_test, h_pred))
    return  metrics.accuracy_score(y_test, h_pred_share_test)


In [5]:
def train_model(t_size):
    # 1. TRAIN THE RANDOM FOREST MODEL

    # 1.1 Import the data sets

    # first the manual visual identifications (true values)
    visual = pd.read_excel(f'{root_dir}/Imports/visual_classification_2020_2021_05_07.xlsx').set_index(['road_id', 'subroad_id'])
    visual[visual['upgrade'] != 1] = 0  # replace unclear and NaN with 0

    # followed by the analysis data
    def import_analysis(data_fp):
        data = pd.read_excel(data_fp,  header=[0,1], index_col=[0]).set_index(
            [('road_id', 'Unnamed: 1_level_1'), ('subroad_id', 'Unnamed: 2_level_1')])
        data.index.names = ['road_id', "subroad_id"]
        data.columns.names = [None, None]
        data = data.drop([('upgrade', 'bright_B4_diff'), ('upgrade', 'bright_B4')], axis=1)
        return data

    data = import_analysis(data_fp)

    X = get_regressors(data)
    y = visual['upgrade'].astype('int')  # Labels

    # check for NaN and drop
    nan_index = X[X.isnull().any(1)].index
    X = X.drop(nan_index, axis=0)
    y = y.drop(nan_index, axis=0)

    seed = 5350
    # Split dataset into training set and test set
    #     - argument test_size = 0.3: 70% training and 30% test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=t_size, random_state=seed)

    ## HERE COMES THE OPTIMIZATION

    # Create the parameter grid based on the results of random search
    param_grid = {
        'n_estimators': [20, 50, 100], # number of trees in the forest
        'max_features': list(range(2, (len(list(get_regressors(X_train).columns))+1))),  # 7 features in the model 7+1 = 8
        'max_depth': [20, 50, 100], # max number of levels in each tree
        'bootstrap': [True, False],
        'min_samples_leaf': [2, 4, 6],
        'min_samples_split': [2, 3, 4],
    }

    rf = RandomForestClassifier(random_state=5350)

    # Instantiate the grid search model
    grid_search = GridSearchCV(estimator = rf, param_grid = param_grid,
                               cv = 5, n_jobs = -1, verbose = 1)

    # grid_search = RandomizedSearchCV(estimator = rf, param_distributions = param_grid,
    #                                n_iter = 100, cv = 3, verbose=1, random_state=5350, n_jobs = -1)

    grid_search.fit(X_train, y_train)

    print(grid_search.best_params_)
    return grid_search.best_params_

In [6]:

# results
best_params = {'bootstrap': True, 'max_depth': 20, 'max_features': 5, 'min_samples_leaf': 6, 'min_samples_split': 2, 'n_estimators': 100}
#train_model(0.1)

In [7]:
def eval_model(fp, t_size):

    # 1. TRAIN THE RANDOM FOREST MODEL

    # 1.1 Import the data sets

    # first the manual visual identifications (true values)
    visual = pd.read_excel(f'{root_dir}/Imports/visual_classification_2020_2021_05_07.xlsx').set_index(['road_id', 'subroad_id'])
    visual[visual['upgrade'] != 1] = 0  # replace unclear and NaN with 0

    # followed by the analysis data
    def import_analysis(data_fp):
        data = pd.read_excel(data_fp,  header=[0,1], index_col=[0]).set_index(
            [('road_id', 'Unnamed: 1_level_1'), ('subroad_id', 'Unnamed: 2_level_1')])
        data.index.names = ['road_id', "subroad_id"]
        data.columns.names = [None, None]
        data = data.drop([('upgrade', 'bright_B4_diff'), ('upgrade', 'bright_B4')], axis=1)
        return data


    data = import_analysis(fp)


    X = get_regressors(data)
    y = visual['upgrade'].astype('int')  # Labels

    # check for NaN and drop
    nan_index = X[X.isnull().any(1)].index
    X = X.drop(nan_index, axis=0)
    y = y.drop(nan_index, axis=0)

    seed = 5350
    # Split dataset into training set and test set
    #     - argument test_size = 0.3: 70% training and 30% test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=t_size, random_state=seed)


    # Create a Gaussian Classifier
    clf = RandomForestClassifier(**best_params, random_state=seed)


    # Train the model using the training sets y_pred=clf.predict(X_test)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print("Random Forest Model Accuracy:", metrics.accuracy_score(y_test, y_pred))
    return metrics.accuracy_score(y_test, y_pred)

In [8]:
data_fp = f"{root_dir}/Imports/analysis_2020_2021_05_07_pct_10.xlsx" # a copy taken from Exports/Roads/composites2
#eval_heuristic(data_fp, 0.1)
#eval_model(data_fp, 0.1)

In [ ]:

output = []
for thresh in list(np.array(range(0, 21))*5):
    analyze_roads(thresh)
    data_fp = f"{root_dir}/Exports/Roads/composites_2/analysis_2020_2021_05_07.xlsx"
    best_params = train_model(0.1)
    h = eval_heuristic(data_fp, 0.1)
    m = eval_model(data_fp, 0.1)
    output.append({'relative_thresh': thresh, 'heuristic': h, 'model': m, 'params': best_params})
    print("COMPLETED", thresh)
df = pd.DataFrame(output)
df

skipping ('B4_mean', 'upg_50')
skipping ('B8_mean', 'upg_50')


C:\Users\Jeffrey\AppData\Local\Programs\Python\Python38-32\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
C:\Users\Jeffrey\AppData\Local\Programs\Python\Python38-32\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
C:\Users\Jeffrey\AppData\Local\Programs\Python\Python38-32\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
C:\Users\Jeffrey\AppData\Local\Programs\Python\Python38-32\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted mul

HURRA! We finished spreadsheet roads_2020_2021_05_07.xlsx
Fitting 5 folds for each of 1134 candidates, totalling 5670 fits
{'bootstrap': True, 'max_depth': 20, 'max_features': 8, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}
MAX ACCURACY FIT TRAINING DATA 0.8266331658291457
THRESH IS [0.09]
Heuristic Model Accuracy: 0.8222222222222222
Random Forest Model Accuracy: 0.9111111111111111
COMPLETED 0
skipping ('B4_mean', 'upg_50')
skipping ('B8_mean', 'upg_50')


C:\Users\Jeffrey\AppData\Local\Programs\Python\Python38-32\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
C:\Users\Jeffrey\AppData\Local\Programs\Python\Python38-32\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
C:\Users\Jeffrey\AppData\Local\Programs\Python\Python38-32\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
C:\Users\Jeffrey\AppData\Local\Programs\Python\Python38-32\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted mul

HURRA! We finished spreadsheet roads_2020_2021_05_07.xlsx
Fitting 5 folds for each of 1134 candidates, totalling 5670 fits


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Jeffrey\AppData\Local\Programs\Python\Python38-32\lib\site-packages\IPython\core\interactiveshell.py", line 3427, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-b2040ea3879f>", line 5, in <module>
    best_params = train_model(0.1)
  File "<ipython-input-5-c5e648c2499e>", line 55, in train_model
    grid_search.fit(X_train, y_train)
  File "C:\Users\Jeffrey\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\utils\validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "C:\Users\Jeffrey\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\model_selection\_search.py", line 841, in fit
    self._run_search(evaluate_candidates)
  File "C:\Users\Jeffrey\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\model_selection\_search.py", line 1288, in _run_search
    evaluate_candidates(ParameterGrid(self.param_grid))
  File "C:\Users\Je

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Jeffrey\AppData\Local\Programs\Python\Python38-32\lib\site-packages\IPython\core\interactiveshell.py", line 3427, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-b2040ea3879f>", line 5, in <module>
    best_params = train_model(0.1)
  File "<ipython-input-5-c5e648c2499e>", line 55, in train_model
    grid_search.fit(X_train, y_train)
  File "C:\Users\Jeffrey\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\utils\validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "C:\Users\Jeffrey\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\model_selection\_search.py", line 841, in fit
    self._run_search(evaluate_candidates)
  File "C:\Users\Jeffrey\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\model_selection\_search.py", line 1288, in _run_search
    evaluate_candidates(ParameterGrid(self.param_grid))
  File "C:\Users\Je

In [9]:
df.to_excel('heuristic_optimization.xlsx')
df

NameError: name 'df' is not defined

ERROR! Session/line number was not unique in database. History logging moved to new session 1147
